# TF-IDF Predictor with Scikit-learn: Proof of Concept

**Term Frequency-Inverse Document Frequency** (TF-IDF) is an information retrieval (IR) statistic used to determine how important a word is to a document. Simply put, we determine the TF-IDF of a word in a document by counting the occurrences of that word in the document, then dividing it by the number of documents in which it appears. Then, this process is repeated for each word in the document, resulting in a vector indicating the relevance of each word to that document. It's a simple, but effective IR method that was introduced in the 1980s, yet has stood the test of time.

The bread and butter of IR is the process of turning arbitrary-length documents into fixed-length vectors (other such methods include word embeddings, encode-decode model, etc.). Once we have vectors that abstractly represent the semantics of documents, we can compare such vectors using metrics like cosine similarity, i.e. the dot product between two vectors. This principle then forms the basis of our project - *using TF-IDF, we map a small query and large documents into a high-dimensional vector space, then determine which documents are most relevant to the query*. In essence, it is a search engine.

In [12]:
# Update and import dependencies
!pip install -Uqr requirements.txt

# Data science/NLP packages
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", None)

# Local modules
import importlib
import tfidf_predictor, utils_tfidf
for m in [tfidf_predictor, utils_tfidf]:
    importlib.reload(m)

from tfidf_predictor import VectorSimilarity, TfidfPredictor
from utils_tfidf import get_data, get_corpus_labels

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


## Extending SKLearn: VectorSimilarity

In order to perform pairwise comparisons across vectors and find the most similar pairs, we implemented the `VectorSimilarity` class, extending SKLearn's `Estimator` interface. The fitting process of `VectorSimilarity` takes two arrays:

1. An array of float vectors, each with the same dimensionality
1. An array of labels, which can have any type or shape.

Note that both arrays must have equal length, and that the `i`th vector corresponds to the `i`th label for all `i`. Then, when we feed it an input vector, `VectorSimilarity` compares that vector against all of the vectors that it was fitted on, returning the corresponding labels of the `n_best` vectors. A "score" is also returned, which is just the dot product with each of the `n_best` vectors.

In [13]:
"""
Visual representation of vector locations

       /\
        b  ?
        |
        |
< c --------- a >
        |
        |
        d
       \/
"""
X = np.array(
    [[1, 0],
     [0, 1],
     [-1, 0],
     [0, -1]]
)
y = np.array(['a', 'b', 'c', 'd'])

sim_estimator = VectorSimilarity(n_best=10)
sim_estimator = sim_estimator.fit(X, y)

vec_input = np.array([0.5, 1]).reshape(1, -1) # Shape needs to be (1, n)
pred, score = sim_estimator.predict(vec_input)
print('Most similar vectors:\n', pred)
print('Confidence scores:\n', score)

(1, 2) <class 'numpy.ndarray'>
(4, 2)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at /root/amplify-observer-nlp/tfidf_predictor.py (48)[0m
[1m
File "tfidf_predictor.py", line 48:[0m
[1m    def linear_kernel_numba(self, u:np.ndarray, v:np.ndarray):
[1m        assert(u.shape[0] == v.shape[0])
[0m        [1m^[0m[0m

This error may have been caused by the following argument(s):
- argument 0: [1mCannot determine Numba type of <class 'tfidf_predictor.VectorSimilarity'>[0m


## Extending SKLearn: TfidfPredictor

To make similarity predictions on natural language, we use the above `VectorSimilarity` class in tandem with SKLearn's `TfidfVectorizer`, tying the two using SKLearn's `Pipeline` interface. We then wrapped the pipeline in another custom `Estimator`, `TfidfPredictor`, in order to have more control over the inputs and outputs of the model.

The `TfidfVectorizer` takes in natural language documents and performs the following transformations:

1. The document is separated into word tokens that are alphabetical and at least 3 characters long.
1. Stop words such as articles, prepositions, and pronouns are removed.
1. Lemmatization is applied to homogenize the different tenses and cases of each word. For example, "walk", "walks" and "walking" all become "walk".
1. TF-IDF values are calculated for each token.

In [ ]:
basic_corpus = [
    'Bees like to make honey',
    'Bears like to eat honey',
    "Bees don't like bears",
    'Humans are walking around the park'
]
basic_labels = ['a', 'b', 'c', 'd']

tfidf_model = TfidfPredictor(lemmatize='custom')
tfidf_model.fit(basic_corpus, basic_labels, verbose=True)
pred, score = tfidf_model.predict(basic_corpus)
print(pred)
print(score)

## Downloading Data and Fitting TF-IDF Predictor

We now download the training data and store it in a CSV, if it doesn't already exist. The training data is stored in [this S3 bucket](https://s3.console.aws.amazon.com/s3/buckets/amplifyobserverinsights-aoinsightslandingbucket29-5vcr471d4nm5). We extract the columns that are used for our corpus and labels, then feed it into a `TfidfPredictor` to fit the model.

In [ ]:
# Train model
df = get_data(r'./data/training_data.csv', force_redownload=False, verbose=True)[:100]

corpus_col='title_body'
url_col = 'url'
title_col='title'

# corpus = train_df[corpus_col]
# labels = list(zip(train_df[url_col], train_df[title_col]))

corpus, labels = get_corpus_labels(df, corpus_col, [url_col, title_col])

tfidf_model = TfidfPredictor(lemmatize='custom')
tfidf_model.fit(corpus, labels, verbose=True)

### Model Statistics
Below, you can see the number of words in the vocabulary. This number is quite high, but the vectors being compared are "sparse" (i.e. having few non-zero values), so dot product computation is fast in the average case.

In [ ]:
# Model stats
vocab = tfidf_model._vectorizer.get_feature_names()
print('Number of words in vocabulary:', len(vocab))
repo_list = list(set(df['repo']))
print('Available repos:', repo_list)

### Performing inferences

Below, we perform an example query prediction. `TfidfPredictor.predict` returns a list of labels corresponding to the `n_best` vectors that are closest to the query's vector, and a list of scores that each of those labels attained.

In [ ]:
query = 'AmplifySignIn component does not work with password managers or native browser autofill'
pred, score = tfidf_model.predict(query, verbose=True)
print(pred)
print(score)

In [30]:
query = 'DataStore model subscription fails'
pred, score = tfidf_model.predict(query, verbose=True)
print(pred)
print(score)

Prediction took 2.426900863647461 seconds
[[['https://github.com/aws-amplify/amplify-cli/issues/3114'
   'DataStore generates subscriptions for all @model types']
  ['https://github.com/aws-amplify/amplify-cli/issues/1070'
   "AppSync Subscriptions not working based on Parent's ID"]
  ['https://github.com/aws-amplify/amplify-android/issues/1388'
   'Datastore model subscription fails due to timeout under slow network connection']
  ['https://github.com/aws-amplify/amplify-js/issues/5050'
   'Limit of 50 subscriptions reached Datastore']
  ['https://github.com/aws-amplify/amplify-js/issues/4067'
   'POSSIBLE BUG: 401 error with Amplify GraphQL Subscription']
  ['https://github.com/aws-amplify/amplify-js/issues/5648'
   'DataStore keeps searching for non-existent Subscriptions']
  ['https://github.com/aws-amplify/amplify-js/issues/4683'
   'Authed subscriptions not working']
  ['https://github.com/aws-amplify/amplify-cli/issues/1810'
   'GraphQL AppSync Subscriptions not generated by AWS

# TODO: add section showing inspection functions